In [ ]:
#@title Tacotron2 and HiFi-GAN Inference Notebook (now with super-resolution!) { display-mode: "form" }

#@markdown fixed by: justinjohn-03

#@markdown ---

#@markdown Instructions:
#@markdown 1. Click the "Open in playground" button. [or CTRL+S and save it]
#@markdown 2. Now click the play icon (top left of this box). [or Ctrl+Enter]
#@markdown 3. Enter your text at the bottom when it's ready.
#@markdown 4. Remember to end your sentences with punctuation.

#@markdown ---

#@markdown Changing Voice/Model:
#@markdown 1. Select a **character** from the dropdown below.
#@markdown 2. Alternatively, paste a model from [this list](https://docs.google.com/document/d/1xe1Clvdg6EFFDtIkkFwT-NPLRDPvkV4G675SUKjxVRU/edit#heading=h.fyifxhdx9qqz) into the *tacotron_override* field. **This will not sound as good as the characters in the dropdown!**
#@markdown 3. Stop and restart this cell by clicking the stop/play icon in the top-left corner.

#@markdown "Fine-tuned" characters will have better audio quality than
#@markdown ones using the universal model. Overrides always
#@markdown use the universal HiFi-GAN model.

#@markdown If you wish to train your own fine-tuned HiFi-GAN models, 
#@markdown use [this notebook](https://colab.research.google.com/drive/1ume3953K2K-EdNL90vNqPNSWM1KRuwqp).

#@markdown MMI and 48Khz Models are **not** supported in this notebook.

#@markdown ---

#@markdown Contributors:
#@markdown - Cookie ([previous version of this notebook](https://colab.research.google.com/drive/1p5Y6cqVAd9NTnFqQ7M11i4hG7M0DwvU2))
#@markdown - Synthbot (Data preprocessing)
#@markdown - Flutteranon (Original Synthesis notebook)
#@markdown - Clipper (Hosting the /mlp/ dataset and many nights of clipping)
#@markdown - SortAnon (adding HiFi-GAN support and audio super-resolution)
#@markdown - /mlp/

#@markdown ---
#@markdown Config:

#@markdown Restart the code to apply any changes.

#Add new characters here.
#Universal HiFi-GAN (has some robotic noise): 1qpgI41wNXFcH-iKq1Y42JlBC9j0je8PW
character = "Rarity (fine-tuned)" #@param ["Rarity (fine-tuned)", "Twilight (fine-tuned)", "Fluttershy (fine-tuned)", "Discord (fine-tuned)", "Trixie (fine-tuned)", "JC Denton (fine-tuned)", "Rapunzel (fine-tuned)", "Clipper (fine-tuned)", "Ghost (fine-tuned)", "HAL 9000 (fine-tuned)", "Maud (universal)"]
tacotron_override = "" #@param {type:"string"}

NO_ARPABET = False
NO_SUPERRES = False
if tacotron_override != "":
    TACOTRON2_ID = tacotron_override
    HIFIGAN_ID = "1P9yB12dF4FH0nX5Tw6zg9DsuuDa0NDed"
elif character == "Rarity (fine-tuned)":
    TACOTRON2_ID = "1RC8Qx7nY6D5olnhjo3rdAuZnWx6T67bE"
    HIFIGAN_ID = "1GZB_lXgB3f8haerHah76uwfEtg2SovD2"
elif character == "Twilight (fine-tuned)":
    TACOTRON2_ID = "1wQo4D_MANiDQk4POcxGBk_v88LYQdaci"
    HIFIGAN_ID = "1eW818BTWO8tE949mg2T-S0ENCErsgPQM"
elif character == "Fluttershy (fine-tuned)":
    TACOTRON2_ID = "13vDA8gXBKNbq0zL3ujAZ2iH3xPJnPZHq"
    HIFIGAN_ID = "1Cwy_VafEucmSjyrpXc62IIk4CuhMb4Tn"
elif character == "JC Denton (fine-tuned)":
    TACOTRON2_ID = "16WjaRv2mXdJTgRkeBf4TMGR0z679ysbU"
    HIFIGAN_ID = "1U4kB7nZIzSnnj-M5h9MFpI7ukIbnzB4T"
elif character == "Maud (universal)":
    TACOTRON2_ID = "1oUikr-FHKY6eYVJxgE-hLqu46VLeSWkb"
    HIFIGAN_ID = "1P9yB12dF4FH0nX5Tw6zg9DsuuDa0NDed"
elif character == "Rapunzel (fine-tuned)":
    TACOTRON2_ID = "1TZ02SlZOyCG0fojPrWRikLuQ4K6tn79s"
    HIFIGAN_ID = "1vFzcD0N8dfA_3TOIepAm-jkNa3hhTCpP"
elif character == "Discord (fine-tuned)":
    TACOTRON2_ID = "1yF437JXw4gkW4XwX3i86gvSWXaCMHTok"
    HIFIGAN_ID = "1L6Hi4VA3PTjNJOiq5w3hLpOOP9ZDmRj1"
elif character == "Clipper (fine-tuned)":
    TACOTRON2_ID = "1FRGp1fvKG1O4HUu2cN2wFT8LRpTCsxrd"
    HIFIGAN_ID = "1nvTK8Z5cvI6iAB-IoqqfxrM79_VGIDrb"
elif character == "HAL 9000 (fine-tuned)":
    TACOTRON2_ID = "12r9fXK42WexER1pS1q30CEXH02i5X3X_"
    HIFIGAN_ID = "1T2Z7AKYP3ql0p1hfus2DbOuL8c2SYRs8"
elif character == "Trixie (fine-tuned)":
    TACOTRON2_ID = "1N68QnzgoOtwvxZX5E6tG6xLVneO539rI"
    HIFIGAN_ID = "1YDcUDvkEwjBCPRzDsDEflMJJTXPhBFTk"
elif character == "Ghost (fine-tuned)":
    TACOTRON2_ID = "1N7zp-p9UqnnvJWhChRpl5veIrU6aQlow"
    HIFIGAN_ID = "1bt2CjWurVylVTGlLyFkj0DizmjjyAwbq"
    NO_ARPABET = True
    NO_SUPERRES = True
else:
    raise Exception("Unknown character!")

# Check if Initialized
try:
    initialized
except NameError:
    print("Setting up, please wait.\n")
    !pip install tqdm -q
    from tqdm.notebook import tqdm
    with tqdm(total=5, leave=False) as pbar:
        import os
        from os.path import exists, join, basename, splitext
        !pip install gdown --upgrade resampy
        !pip install git+https://github.com/wkentaro/gdown.git
        git_repo_url = 'https://github.com/justinjohn0306/TTS-TT2.git'
        project_name = splitext(basename(git_repo_url))[0]
        if not exists(project_name):
            # clone and install
            !git clone -q --recursive {git_repo_url}
            !git clone -q --recursive https://github.com/SortAnon/hifi-gan
            !pip install -q librosa unidecode
        pbar.update(1) # downloaded TT2 and HiFi-GAN
        import sys
        sys.path.append('hifi-gan')
        sys.path.append(project_name)
        import time
        import matplotlib
        import matplotlib.pylab as plt
        import gdown
        d = 'https://drive.google.com/uc?id='

        %matplotlib inline
        import IPython.display as ipd
        import numpy as np
        import torch
        import json
        from hparams import create_hparams
        from model import Tacotron2
        from layers import TacotronSTFT
        from audio_processing import griffin_lim
        from text import text_to_sequence
        from env import AttrDict
        from meldataset import mel_spectrogram, MAX_WAV_VALUE
        from models import Generator
        from denoiser import Denoiser
        import resampy
        import scipy.signal

        pbar.update(1) # initialized Dependancies

        graph_width = 900
        graph_height = 360
        def plot_data(data, figsize=(int(graph_width/100), int(graph_height/100))):
            %matplotlib inline
            fig, axes = plt.subplots(1, len(data), figsize=figsize)
            for i in range(len(data)):
                axes[i].imshow(data[i], aspect='auto', origin='bottom', 
                            interpolation='none', cmap='inferno')
            fig.canvas.draw()
            plt.show()

        # Setup Pronounciation Dictionary
        !gdown  '1IyTaUW2h135ULs4T3dvGpFU7ZhmALn1_'
        thisdict = {}
        for line in reversed((open('merged.dict.txt', "r").read()).splitlines()):
            thisdict[(line.split(" ",1))[0]] = (line.split(" ",1))[1].strip()

        pbar.update(1) # Downloaded and Set up Pronounciation Dictionary

        def ARPA(text, punctuation=r"!?,.;", EOS_Token=True):
            out = ''
            for word_ in text.split(" "):
                word=word_; end_chars = ''
                while any(elem in word for elem in punctuation) and len(word) > 1:
                    if word[-1] in punctuation: end_chars = word[-1] + end_chars; word = word[:-1]
                    else: break
                try:
                    word_arpa = thisdict[word.upper()]
                    word = "{" + str(word_arpa) + "}"
                except KeyError: pass
                out = (out + " " + word + end_chars).strip()
            if EOS_Token and out[-1] != ";": out += ";"
            return out

        def get_hifigan(MODEL_ID, conf_name):
            # Download HiFi-GAN
            hifigan_pretrained_model = 'hifimodel_' + conf_name
            gdown.download(d+MODEL_ID, hifigan_pretrained_model, quiet=False)
            if not exists(hifigan_pretrained_model):
                raise Exception("HiFI-GAN model failed to download!")

            # Load HiFi-GAN
            conf = os.path.join("hifi-gan", conf_name + ".json")
            with open(conf) as f:
                json_config = json.loads(f.read())
            h = AttrDict(json_config)
            torch.manual_seed(h.seed)
            hifigan = Generator(h).to(torch.device("cuda"))
            state_dict_g = torch.load(hifigan_pretrained_model, map_location=torch.device("cuda"))
            hifigan.load_state_dict(state_dict_g["generator"])
            hifigan.eval()
            hifigan.remove_weight_norm()
            denoiser = Denoiser(hifigan, mode="normal")
            return hifigan, h, denoiser

        # Download character HiFi-GAN
        hifigan, h, denoiser = get_hifigan(HIFIGAN_ID, "config_v1")
        # Download super-resolution HiFi-GAN
        hifigan_sr, h2, denoiser_sr = get_hifigan("14fOprFAIlCQkVRxsfInhEPG0n-xN4QOa", "config_32k")
        pbar.update(1) # Downloaded and Set up HiFi-GAN

        def has_MMI(STATE_DICT):
            return any(True for x in STATE_DICT.keys() if "mi." in x)

        def get_Tactron2(MODEL_ID):
            # Download Tacotron2
            tacotron2_pretrained_model = 'MLPTTS'
            gdown.download(d+MODEL_ID, tacotron2_pretrained_model, quiet=False)
            if not exists(tacotron2_pretrained_model):
                raise Exception("Tacotron2 model failed to download!")
            # Load Tacotron2 and Config
            hparams = create_hparams()
            hparams.sampling_rate = 22050
            hparams.max_decoder_steps = 3000 # Max Duration
            hparams.gate_threshold = 0.25 # Model must be 25% sure the clip is over before ending generation
            model = Tacotron2(hparams)
            state_dict = torch.load(tacotron2_pretrained_model)['state_dict']
            if has_MMI(state_dict):
                raise Exception("ERROR: This notebook does not currently support MMI models.")
            model.load_state_dict(state_dict)
            _ = model.cuda().eval().half()
            return model, hparams

        model, hparams = get_Tactron2(TACOTRON2_ID)
        previous_tt2_id = TACOTRON2_ID

        pbar.update(1) # Downloaded and Set up Tacotron2

        # Extra Info
        def end_to_end_infer(text, pronounciation_dictionary, show_graphs):
            for i in [x for x in text.split("\n") if len(x)]:
                if not pronounciation_dictionary:
                    if i[-1] != ";": i=i+";" 
                else: i = ARPA(i)
                with torch.no_grad(): # save VRAM by not including gradients
                    sequence = np.array(text_to_sequence(i, ['english_cleaners']))[None, :]
                    sequence = torch.autograd.Variable(torch.from_numpy(sequence)).cuda().long()
                    mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)
                    if show_graphs:
                        plot_data((mel_outputs_postnet.float().data.cpu().numpy()[0],
                                alignments.float().data.cpu().numpy()[0].T))
                    y_g_hat = hifigan(mel_outputs_postnet.float())
                    audio = y_g_hat.squeeze()
                    audio = audio * MAX_WAV_VALUE
                    audio_denoised = denoiser(audio.view(1, -1), strength=35)[:, 0]

                    # Resample to 32k
                    audio_denoised = audio_denoised.cpu().numpy().reshape(-1)

                    normalize = (MAX_WAV_VALUE / np.max(np.abs(audio_denoised))) ** 0.9
                    audio_denoised = audio_denoised * normalize
                    wave = resampy.resample(
                        audio_denoised,
                        h.sampling_rate,
                        h2.sampling_rate,
                        filter="sinc_window",
                        window=scipy.signal.windows.hann,
                        num_zeros=8,
                    )
                    wave_out = wave.astype(np.int16)

                    # HiFi-GAN super-resolution
                    wave = wave / MAX_WAV_VALUE
                    wave = torch.FloatTensor(wave).to(torch.device("cuda"))
                    new_mel = mel_spectrogram(
                        wave.unsqueeze(0),
                        h2.n_fft,
                        h2.num_mels,
                        h2.sampling_rate,
                        h2.hop_size,
                        h2.win_size,
                        h2.fmin,
                        h2.fmax,
                    )
                    y_g_hat2 = hifigan_sr(new_mel)
                    audio2 = y_g_hat2.squeeze()
                    audio2 = audio2 * MAX_WAV_VALUE
                    audio2_denoised = denoiser(audio2.view(1, -1), strength=35)[:, 0]

                    # High-pass filter, mixing and denormalizing
                    audio2_denoised = audio2_denoised.cpu().numpy().reshape(-1)
                    b = scipy.signal.firwin(
                        101, cutoff=10500, fs=h2.sampling_rate, pass_zero=False
                    )
                    y = scipy.signal.lfilter(b, [1.0], audio2_denoised)
                    y *= superres_strength
                    y_out = y.astype(np.int16)
                    y_padded = np.zeros(wave_out.shape)
                    y_padded[: y_out.shape[0]] = y_out
                    sr_mix = wave_out + y_padded
                    sr_mix = sr_mix / normalize

                    print("")
                    ipd.display(ipd.Audio(sr_mix.astype(np.int16), rate=h2.sampling_rate))
    from IPython.display import clear_output
    clear_output()
    initialized = "Ready"

if previous_tt2_id != TACOTRON2_ID:
    print("Updating Models")
    model, hparams = get_Tactron2(TACOTRON2_ID)
    hifigan, h, denoiser = get_hifigan(HIFIGAN_ID, "config_v1")
    previous_tt2_id = TACOTRON2_ID

pronounciation_dictionary = False #@param {type:"boolean"}
# disables automatic ARPAbet conversion, useful for inputting your own ARPAbet pronounciations or just for testing
show_graphs = True #@param {type:"boolean"}
max_duration = 25 #@param {type:"integer"}
model.decoder.max_decoder_steps = max_duration * 100
stop_threshold = 0.5 #@param {type:"number"}
model.decoder.gate_threshold = stop_threshold
superres_strength = 5.0 #@param {type:"number"}

#@markdown ---

print(f"Current Config:\npronounciation_dictionary: {pronounciation_dictionary}\nshow_graphs: {show_graphs}\nmax_duration (in seconds): {max_duration}\nstop_threshold: {stop_threshold}\nsuperres_strength: {superres_strength}\n\n")

time.sleep(1)
print("Enter/Paste your text.")
contents = []
while True:
    try:
        print("-"*50)
        line = input()
        if line == "":
            continue
        end_to_end_infer(line, not pronounciation_dictionary, show_graphs)
    except EOFError:
        break
    except KeyboardInterrupt:
        print("Stopping...")
        break